In [4]:
import matplotlib
matplotlib.use('Agg')

%load_ext autoreload
%autoreload 2

%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
import os

import glob2

# visualize results module
from Visualize import Visualize

from utility_DLC import *


# 
#data_dir = '/media/cat/4TBSSD/yuki/'
data_dir = '/media/cat/4TBSSD/yuki/'


labels = ['left_paw', 
'right_paw',
'nose',
'jaw',
'right_ear',
'tongue',
'lever']



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [2]:
##########################
####### INITIALIZE #######
##########################
# LEVER PULL
vis = Visualize()

# lever-related data
vis.main_dir = data_dir
vis.random_flag = False  # shuffle data to show baseline

# 
vis.window = 15
vis.lockout_window = 10
vis.lockout = False

vis.pca_var = 0.95
vis.pca_flag = True


In [24]:
#########################################
######## PLOT SVM CHOICE SESSION ########
#########################################
vis.significance = 0.05
vis.linewidth=10

#
vis.smooth_window = 10
vis.lockout = False
vis.xvalidation = 10
vis.sliding_window = 30

# 
vis.window = 15
vis.cbar_thick = 0.05
vis.alpha = 1.0
    
# 
vis.min_trials = 10
# 
vis.fig=plt.figure(figsize=(10,20))
vis.ctr_plot = 0
vis.ax = plt.subplot(1,1,1)
    
    

# 
vis.animal_id= 'IA1'
session = 'Feb1_'

print ("   session: ", session)
vis.session_id = session
vis.cbar_offset = 0

############################

codes = ['left_paw','right_paw','jaw']
#code = 'left_paw'
cmaps = ['Reds_r',"Blues_r","Greens_r"]
clrs = ['red','blue','green']
vis.show_title=False
vis.show_legend=False
vis.legend_location = 2
vis.cbar=False
vis.show_EDT = False
vis.edt_offset = 0
vis.linewidth = 1

# alignment tools
vis.show_all_times = True
vis.shift_flag = True
vis.shift = 0

# 
vis.plotted= False # flag to indicate at least 1 plot was made
vis.n_trials_plotting = []
for ctr,code in enumerate(codes):
    vis.ctr = ctr
    vis.code = code

    vis.cbar_offset=0.05*ctr
    vis.cmap = cmaps[ctr]
    vis.title = vis.code
    vis.title_offset = .35
    vis.plot_significant(clrs[ctr],vis.code)
    vis.cbar=False
    
    #vis.ax.set_xticks([])

if vis.plotted:
    plt.title(str(vis.n_trials_plotting))
    vis.ctr_plot+=1

        
   # if ctrs>30:
   #     break
plt.suptitle("Animal ID: "+ vis.animal_id, fontsize=20)

if False:
    plt.savefig('/home/cat/'+vis.animal_id+'.png')
    plt.close()
else:
    plt.show()
#plt.xlim(-5,0)
#plt.show()





   session:  Feb1_
SHIFT Loaded:  4.605
... shift applied:  4.605
LOADED: DATA (871, 10)
 post fname:  (871, 10)
post process:  (862, 10)
self n trials:  73
self.data:  (862, 10)
Self sig save:  (862,)
data into multi-hypothesis tes: (862,)
Final sig:  (1, 862)
earliest:  0  in sec:  -28.733333333333334
vmin, vmax;  0.0 0.05 ticks:  [0.         0.01666667 0.03333333 0.05      ]
/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/shift.txt
SHIFT Loaded:  4.605
... shift applied:  4.605
LOADED: DATA (871, 10)
 post fname:  (871, 10)
post process:  (862, 10)
self n trials:  60
self.data:  (862, 10)
Self sig save:  (862,)
data into multi-hypothesis tes: (862,)
Final sig:  (1, 862)
earliest:  0  in sec:  -28.733333333333334
vmin, vmax;  0.0 0.05 ticks:  [0.         0.01666667 0.03333333 0.05      ]
/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/shift.txt
SHIFT Loaded:  4.605
... shift applied:  4.605
LOADED: DATA (871, 10)
 post fname:  (871, 10)
post process:  (862, 10)
self n trial

In [153]:
# manually add labels from DLC
from tqdm import tqdm, trange
import scipy

def clean_traces(data,
                smooth_window = 10):
    
    # 
    for k in trange(data.shape[0]):
        temp = data[k]
        probs = temp[:,2]
        
        threshold = 0.5
        idx = np.where(probs<threshold)[0]
        x = temp[:,0]
        y = temp[:,1]
        x[idx] = np.nan
        y[idx] = np.nan
        
        # 

        for i in range(10000):
            idx = np.where(np.isnan(x))[0]
            #print (idx.shape)
            if idx.shape[0]==0:
                break

            if idx[0]==0:
                idx=idx[1:]
                
            x[idx] = x[idx-1]
            y[idx] = y[idx-1]

        x = scipy.ndimage.median_filter(x, size=smooth_window)
        y = scipy.ndimage.median_filter(y, size=smooth_window)
        #print (x.shape, y.shape, data.shape)
        
        data[k,:,0]=x
        data[k,:,1]=y
    return data
            


# 
import glob

fnames = np.loadtxt('/media/cat/4TBSSD/yuki/IA1/tif_files/sessions.txt',dtype='str')
#root_dir = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/'
for root_dir in fnames:
    
    try:
        fname_h5 = glob.glob(root_dir + '/*.h5')[0]
        fname_npy = fname_h5.replace('h5','npy')
    except:
        print("video file not found: ", root_dir)
        continue
    
    fname_out = fname_npy.replace('.npy','_clean.npy')

    if os.path.exists(fname_out)==False:
        print ('processing: ', root_dir)
        traces_original = np.load(fname_npy)
        if True:
            traces_original[:,:,0] = np.int32(traces_original[:,:,0])
            traces_original[:,:,1] = np.int32(traces_original[:,:,1])

        print (traces_original.shape)

        # plt.plot(traces_original[0,:,0])
        smooth_window = 3
        traces = clean_traces(traces_original, smooth_window)
        print (traces.shape)

        np.save(fname_out,traces)
        print ('')
# process traces


video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb18_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb19_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb22_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb23_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb24_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb25_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb26_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb29_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Mar1_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Mar2_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Mar3_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_Mar4_30Hz
video file not found:  /media/cat/4TBSSD/yuki/IA1/tif_files/IA1am_Mar7_30Hz
vide

In [29]:
# triggers load

def plot_velocities(traces, movements, window):
    
    # 
    t=np.linspace(-window, window,window*2*15-1)

    
    fig=plt.figure()
    ax1=plt.subplot(111)
    mean_vel=[]
    for k in trange(movements.shape[0]):
        t_start = int(movements[k]-window)*15
        t_end = int(movements[k]+window)*15

        temp = traces[feature_selected,t_start:t_end,:2]

        if True:
            ax1=plt.subplot(221)
            vel = np.sqrt((temp[1:,0]-temp[:-1,0])**2+
                          (temp[1:,1]-temp[:-1,1])**2)

            mean_vel.append(vel)
            plt.plot(t,vel,c='black',linewidth=2,alpha=.1)

    # complete ax1 plots
    mean_vel = np.array(mean_vel).mean(0)
    ax1.plot(t,mean_vel,c='black',linewidth=5,alpha=1)
    ax1.plot([0,0],[0.1,250],'--',linewidth=5,c='black')
    ax1.set_ylim(0.1,250)
    ax1.set_xlim(t[0],t[-1])
    ax1.semilogy()




def get_positions(movements,
                   traces,
                   colors,
                   plotting=False):
    
    padding_x = 20
    padding_y = 20

    if plotting:
        fig=plt.figure()
        ax1=plt.subplot(111)
    
    # 
    mean_trace = []
    for k in trange(movements.shape[0]):
        t_start = int(movements[k]-window)*15
        t_end = int(movements[k]+window)*15

        temp = traces[feature_selected,t_start:t_end,:2]
        
        temp = temp - temp[temp.shape[0]//2]
        temp[:,0] = scipy.ndimage.median_filter(temp[:,0], size=3)
        temp[:,1] = scipy.ndimage.median_filter(temp[:,1], size=3)

        if plotting:
            if True:
                for p in range(1, temp.shape[0]-1,1):
                    plt.plot([temp[p-1,0],temp[p,0]],
                             [temp[p-1,1],temp[p,1]], 
                             c=colors[p],
                             linewidth=3,
                             alpha=.1)
            else:
                plt.plot(temp[:,0], temp[:,1], 
                         c=colors[k],alpha=.4)


            plt.scatter(temp[:,0],temp[:,1], 
                         c=colors, 
                        s=100, alpha=.1)
        
        #print (temp.shape, t_end, t_start)
        if temp.shape[0]==(t_end-t_start):
            mean_trace.append(temp)
    
    # complete ax2 plots
    if plotting:
        ax1.set_xlim(ax1.get_xlim()[0]-padding_x,
                     ax1.get_xlim()[1]+padding_x)
        ax1.set_ylim(ax1.get_ylim()[0]-padding_y,
                     ax1.get_ylim()[1]+padding_y)


    mean_trace = np.array(mean_trace)


    return np.array(mean_trace)

# 
def plot_average_positions(mean_trace, color):
    
    #fig = plt.figure()
    ax1=plt.subplot(111)
    #
    mean_trace = mean_trace.mean(0)
    mean_trace[:,0] = scipy.ndimage.median_filter(mean_trace[:,0], size=10)
    mean_trace[:,1] = scipy.ndimage.median_filter(mean_trace[:,1], size=10)

    # 
    for p in range(1, mean_trace.shape[0]-1,1):
        ax1.plot([mean_trace[p-1,0],mean_trace[p,0]],
                 [mean_trace[p-1,1],mean_trace[p,1]], 
                 c=color,
                 linewidth=5, alpha=.1)

    ax1.scatter(mean_trace[:,0],mean_trace[:,1], 
                #c=colors,alpha=1)
                c=color,alpha=.1)

    ax1.set_xlim(np.min(mean_trace[:,0])-1,
                 np.max(mean_trace[:,0])+1)
    ax1.set_ylim(np.min(mean_trace[:,1])-1,
                 np.max(mean_trace[:,1])+1)



import glob
window = 5

n = int(window)*15*2
colors = plt.cm.cool(np.linspace(0,1,n))


colors_all = plt.cm.viridis(np.linspace(0,1,20))

fnames = np.loadtxt('/media/cat/4TBSSD/yuki/IA1/tif_files/sessions.txt',dtype='str')
#root_dir = '/media/cat/4TBSSD/yuki/IA1/tif_files/IA1pm_Feb1_30Hz/'
ctr=0
for root_dir in fnames:
    
    try:
        fname_h5 = glob.glob(root_dir + '/*.h5')[0]
        fname_npy = fname_h5.replace('h5','npy')
    except:
        continue
        
    feature_selected = 0
    fname_feature = glob.glob(root_dir + "/*"+labels[feature_selected]+"_trial*selected.txt")
    movements = np.loadtxt(fname_feature[0])
    print (movements.shape)


    traces = np.load(fname_npy)

    # 
    if False:
        plot_velocities(traces, 
                    movements, 
                    window)

    mean_trace = get_positions(movements,
                       traces,
                       colors)


    plot_average_positions(mean_trace,
                           colors_all[ctr])

    ctr+=1
    #break

plt.suptitle(labels[feature_selected],fontsize=20)


if False:
    plt.savefig('/home/cat/'+labels[feature_selected]+'.png',dpi=300)
    plt.close()
else:
    plt.show()


100%|██████████| 73/73 [00:00<00:00, 5568.91it/s]

(73,)



*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
100%|██████████| 54/54 [00:00<00:00, 5626.30it/s]
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
100%|██████████| 48/48 [00:00<00:00, 5589.61it/s]


(54,)
(48,)


*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
100%|██████████| 74/74 [00:00<00:00, 5822.14it/s]
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
100%|██████████| 84/84 [00:00<00:00, 5644.19it/s]


...

*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
100%|██████████| 49/49 [00:00<00:00, 5759.95it/s]
*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
100%|██████████| 55/55 [00:00<00:00, 5529.67it/s]


(49,)
(55,)


*c* argument looks like a single numeric RGB or RGBA sequence, which should be avoided as value-mapping will have precedence in case its length matches with *x* & *y*.  Please use the *color* keyword-argument or provide a 2D array with a single row if you intend to specify the same RGB or RGBA value for all points.
100%|██████████| 34/34 [00:00<00:00, 5407.49it/s]

(34,)


IndexError: index 20 is out of bounds for axis 0 with size 20

In [50]:
#############################################
######### MAKE A MOVIE WITH DLC TRACES ######
#############################################

# #
# fname_traces = ''
# traces = np.load(fname_traces)
print (traces.shape)

# 
fname_video = '/media/cat/4TBSSD/yuki/IA1/vids/prestroke/IA1pm_Feb1_30Hz.mp4'
movements = np.zeros((traces.shape[0],traces.shape[1]),'int32')

# 
start = 0*15+1
end = start+60*15
make_video_dlc(traces_original,
               movements,
               fname_video,
               start,
               end)

  3%|▎         | 30/900 [00:00<00:02, 293.48it/s]

(7, 20059, 3)
Traces:  (7, 20059, 3)
loadin gmovie:  /media/cat/4TBSSD/yuki/IA1/vids/prestroke/IA1pm_Feb1_30Hz.mp4
Frame size read:  (360, 640, 3)


100%|██████████| 900/900 [00:03<00:00, 286.22it/s]
